# Agent: Validation

이 튜토리얼을 통해 다음을 배울 수 있다:

- LLM 출력의 불확실성과 유효성 검사의 필요성을 이해한다.
- Pydantic을 활용한 스키마를 정의하고 검증한다.
- Instructor 라이브러리로 구조화된 출력을 강제한다.
- 복잡한 데이터 구조의 유효성 검사를 구현한다.

## 1. 환경 설정

### 1.1 필요한 라이브러리 설치
라이브러리 설명은 다음과 같다:
- **pydantic**: 데이터 검증 및 스키마 정의 라이브러리다.
- **instructor**: OpenAI API를 확장하여 구조화된 출력을 지원하는 라이브러리다.

In [9]:
%pip install -q instructor

Note: you may need to restart the kernel to use updated packages.


### 1.2 환경 설정

In [10]:
from dotenv import load_dotenv
import os

load_dotenv()

True

### 1.3 필요한 라이브러리 가져오기

In [11]:
from openai import OpenAI
from pydantic import BaseModel, Field
import instructor

## 2. 기본 유효성 검사

### 2.1 Pydantic 모델 정의
먼저 LLM 출력의 예상 구조를 Pydantic 모델로 정의한다.

In [12]:
class TaskResult(BaseModel):
    """작업 정보를 담는 데이터 모델이다."""
    task: str = Field(..., description="작업 내용")
    completed: bool = Field(..., description="완료 여부")
    priority: int = Field(..., description="우선순위")

### 2.2 Instructor로 OpenAI 클라이언트 패치
Instructor는 OpenAI 클라이언트를 확장하여 `response_model` 인자를 지원하게 만든다.

In [13]:
MODEL = "gpt-4o-mini"

client = instructor.patch(OpenAI())

### 2.3 구조화된 출력 생성

In [15]:
def structured_intelligence(prompt: str) -> TaskResult:
    """구조화된 출력을 생성하는 지능형 함수다."""
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "사용자 입력에서 작업 정보를 추출한다."},
            {"role": "user", "content": prompt},
        ],
        response_model=TaskResult, # 이 부분이 핵심이다
    )
    return response

# 사용 예시
result = structured_intelligence(
    "금요일까지 프로젝트 발표를 완료해야 하는데, 우선순위가 높다."
)

print("=== 구조화된 출력 ===")
print(result.model_dump_json(indent=2))
print(f"\n작업: {result.task}")
print(f"완료 여부: {result.completed}")
print(f"우선순위: {result.priority}")

=== 구조화된 출력 ===
{
  "task": "프로젝트 발표 완료",
  "completed": false,
  "priority": 1
}

작업: 프로젝트 발표 완료
완료 여부: False
우선순위: 1


## 3. 고급 유효성 검사

### 3.1 Field를 활용한 세밀한 검증
`Field`를 사용하면 값의 범위, 길이, 패턴 등을 검증할 수 있다.

In [16]:
from pydantic import BaseModel, Field
from typing import List

class UserProfile(BaseModel):
    """사용자 프로필 모델이다."""
    name: str = Field(..., min_length=2, max_length=50, description="사용자 이름")
    age: int = Field(..., ge=0, le=150, description="나이 (0-150)")
    email: str = Field(..., pattern=r'^[\w\.-]+@[\w\.-]+\.\w+$', description="이메일 주소")
    interests: List[str] = Field(..., min_length=1, max_length=10, description="관심사 목록")
    score: float = Field(..., ge=0.0, le=100.0, description="점수 (0-100)")

def extract_user_profile(text: str) -> UserProfile:
    """텍스트에서 사용자 프로필을 추출한다."""
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "텍스트에서 사용자 정보를 추출한다."},
            {"role": "user", "content": text}
        ],
        response_model=UserProfile,
    )
    return response

# 사용 예시
profile_text = """
저는 김철수이고 28살입니다. 
이메일은 chulsoo.kim@example.com이에요.
프로그래밍, 독서, 운동에 관심이 있습니다.
최근 평가에서 85.5점을 받았어요.
"""

profile = extract_user_profile(profile_text)
print(f"이름: {profile.name}")
print(f"나이: {profile.age}")
print(f"이메일: {profile.email}")
print(f"관심사: {', '.join(profile.interests)}")
print(f"점수: {profile.score}")

이름: 김철수
나이: 28
이메일: chulsoo.kim@example.com
관심사: 프로그래밍, 독서, 운동
점수: 85.5


### 3.2 중첩된 모델 검증
복잡한 데이터 구조도 검증할 수 있다.

In [17]:
from pydantic import BaseModel, Field
from typing import List, Optional

class Address(BaseModel):
    """주소 정보 모델이다."""
    street: str = Field(..., description="도로명")
    city: str = Field(..., description="도시")
    postal_code: str = Field(..., pattern=r'^\d{5}$', description="우편번호 (5자리)")

class OrderItem(BaseModel):
    """주문 항목 모델이다."""
    product_name: str = Field(..., description="제품명")
    quantity: int = Field(..., ge=1, description="수량")
    price: float = Field(..., ge=0, description="단가")

class Order(BaseModel):
    """주문 정보 모델이다."""
    order_id: str = Field(..., description="주문 번호")
    customer_name: str = Field(..., description="고객 이름")
    items: List[OrderItem] = Field(..., min_length=1, description="주문 항목 목록")
    shipping_address: Address = Field(..., description="배송 주소")
    total_amount: float = Field(..., ge=0, description="총 금액")
    order_date: str = Field(..., description="주문 날짜 (YYYY-MM-DD 형식)")
    notes: Optional[str] = Field(None, description="추가 메모")

def parse_order(order_text: str) -> Order:
    """주문 정보를 파싱한다."""
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "주문 정보를 구조화된 형식으로 추출한다."},
            {"role": "user", "content": order_text}
        ],
        response_model=Order,
    )
    return response

# 사용 예시
order_text = """
주문번호: ORD-2025-001
고객: 이영희
주문 내용:
- 무선 마우스 2개, 각 25000원
- 키보드 1개, 89000원
총 금액: 139000원
배송지: 서울시 강남구 테헤란로 123, 우편번호 06234
주문일: 2025-01-15
특이사항: 배송 전 연락 부탁드립니다.
"""

order = parse_order(order_text)
print(f"주문번호: {order.order_id}")
print(f"고객: {order.customer_name}")
print(f"주문 항목: {len(order.items)}개")
for item in order.items:
    print(f"  - {item.product_name}: {item.quantity}개 x {item.price:,.0f}원")
print(f"배송지: {order.shipping_address.city} {order.shipping_address.street}")
print(f"총액: {order.total_amount:,.0f}원")

주문번호: ORD-2025-001
고객: 이영희
주문 항목: 2개
  - 무선 마우스: 2개 x 25,000원
  - 키보드: 1개 x 89,000원
배송지: 서울시 강남구 테헤란로 123
총액: 139,000원


### 3.3 열거형(Enum)을 사용한 값 제한
특정 값만 허용하도록 제한할 수 있다.

In [18]:
from enum import Enum
from pydantic import BaseModel, Field

class Priority(str, Enum):
    """우선순위 열거형이다."""
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"
    URGENT = "urgent"

class Status(str, Enum):
    """작업 상태 열거형이다."""
    TODO = "todo"
    IN_PROGRESS = "in_progress"
    REVIEW = "review"
    DONE = "done"

class Task(BaseModel):
    """작업 모델이다."""
    title: str = Field(..., description="작업 제목")
    description: str = Field(..., description="작업 설명")
    priority: Priority = Field(..., description="우선순위")
    status: Status = Field(Status.TODO, description="현재 상태")
    assignee: str = Field(..., description="담당자")

def create_task(task_description: str) -> Task:
    """작업 설명에서 구조화된 작업을 생성한다."""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "작업 정보를 추출한다."},
            {"role": "user", "content": task_description}
        ],
        response_model=Task,
    )
    return response

# 사용 예시
task = create_task(
    "긴급하게 데이터베이스 백업을 해야 해. 김철수 담당으로 부탁해."
)

print(f"제목: {task.title}")
print(f"우선순위: {task.priority.value}")
print(f"상태: {task.status.value}")
print(f"담당자: {task.assignee}")

제목: 데이터베이스 백업
우선순위: urgent
상태: todo
담당자: 김철수


## 4. 실전 활용 예제

### 4.1 이메일 분류 시스템
이메일을 자동으로 분류하고 중요 정보를 추출한다.

In [19]:
from pydantic import BaseModel, Field
from typing import List
from enum import Enum

class EmailCategory(str, Enum):
    """이메일 카테고리다."""
    URGENT = "urgent"
    WORK = "work"
    PERSONAL = "personal"
    SPAM = "spam"
    NEWSLETTER = "newsletter"

class EmailAnalysis(BaseModel):
    """이메일 분석 결과 모델이다."""
    subject: str = Field(..., description="이메일 제목")
    category: EmailCategory = Field(..., description="이메일 카테고리")
    sender_name: str = Field(..., description="발신자 이름")
    key_points: List[str] = Field(..., min_length=1, max_length=5, description="핵심 요점")
    requires_response: bool = Field(..., description="답장 필요 여부")
    urgency_score: int = Field(..., ge=1, le=10, description="긴급도 (1-10)")
    suggested_action: str = Field(..., description="권장 조치")

def analyze_email(email_content: str) -> EmailAnalysis:
    """이메일을 분석한다."""
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "이메일을 분석하고 중요 정보를 추출한다."},
            {"role": "user", "content": email_content}
        ],
        response_model=EmailAnalysis,
    )
    return response

# 사용 예시
email = """
발신: 프로젝트 매니저 박민수
제목: [긴급] 내일 오전 클라이언트 미팅 준비

안녕하세요,

내일 오전 10시 클라이언트 미팅이 예정되어 있습니다.
다음 사항을 준비해 주세요:
1. 최신 프로젝트 진행 상황 보고서
2. 예산 집행 현황
3. 향후 일정 계획

늦어도 오늘 오후 5시까지 자료를 공유 부탁드립니다.

감사합니다.
"""

result = analyze_email(email)
print(f"카테고리: {result.category.value}")
print(f"긴급도: {result.urgency_score}/10")
print(f"답장 필요: {'예' if result.requires_response else '아니오'}")
print("\n핵심 요점:")
for point in result.key_points:
    print(f"  - {point}")
print(f"\n권장 조치: {result.suggested_action}")

카테고리: urgent
긴급도: 9/10
답장 필요: 예

핵심 요점:
  - 내일 오전 10시 클라이언트 미팅
  - 최신 프로젝트 진행 상황 보고서 준비
  - 예산 집행 현황 준비
  - 향후 일정 계획 준비
  - 오늘 오후 5시까지 자료 공유

권장 조치: 자료를 오늘 오후 5시까지 준비하여 공유하기


### 4.2 제품 리뷰 감정 분석
리뷰를 분석하여 구조화된 인사이트를 추출한다.

In [21]:
from pydantic import BaseModel, Field
from typing import List

class Sentiment(str, Enum):
    """감정 분류다."""
    VERY_POSITIVE = "very_positive"
    POSITIVE = "positive"
    NEUTRAL = "neutral"
    NEGATIVE = "negative"
    VERY_NEGATIVE = "very_negative"

class ReviewAnalysis(BaseModel):
    """리뷰 분석 결과 모델이다."""
    overall_sentiment: Sentiment = Field(..., description="전반적인 감정")
    rating: int = Field(..., ge=1, le=5, description="예상 평점 (1-5)")
    pros: List[str] = Field(..., description="장점 목록")
    cons: List[str] = Field(..., description="단점 목록")
    main_topics: List[str] = Field(..., description="주요 언급 주제")
    would_recommend: bool = Field(..., description="추천 의향")
    summary: str = Field(..., max_length=200, description="리뷰 요약")

def analyze_review(review_text: str) -> ReviewAnalysis:
    """제품 리뷰를 분석한다."""
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "제품 리뷰를 상세히 분석한다."},
            {"role": "user", "content": review_text}
        ],
        response_model=ReviewAnalysis,
    )
    return response

# 사용 예시
review = """
이 무선 이어폰을 3개월간 사용해 봤는데 전반적으로 만족스럽습니다.
음질이 깨끗하고 배터리도 하루 종일 사용할 수 있어요.
노이즈 캔슬링 기능도 훌륭합니다.

다만 케이스가 좀 크고 무거운 편이라 휴대성은 아쉽습니다.
가격도 경쟁 제품에 비해 조금 비싼 편이에요.

그래도 음질을 중시하는 분들께는 추천하고 싶습니다.
"""

analysis = analyze_review(review)
print(f"감정: {analysis.overall_sentiment.value}")
print(f"평점: {analysis.rating}/5")
print(f"추천: {'예' if analysis.would_recommend else '아니오'}")

print("\n장점:")
for pro in analysis.pros:
    print(f"  {pro}")

print("\n단점:")
for con in analysis.cons:
    print(f"  {con}")

print(f"\n요약: {analysis.summary}")

감정: positive
평점: 4/5
추천: 예

장점:
  음질이 깨끗하다
  배터리 수명이 길다
  노이즈 캔슬링 기능이 훌륭하다

단점:
  케이스가 크고 무겁다
  가격이 비싸다

요약: 3개월 사용 후 전반적으로 만족하며, 음질과 배터리 성능이 뛰어나지만 케이스 크기와 가격이 아쉽다.


### 4.3 회의록 구조화
회의 내용을 자동으로 정리하고 구조화한다.

In [22]:
from pydantic import BaseModel, Field
from typing import List

class ActionItem(BaseModel):
    """실행 항목 모델이다."""
    task: str = Field(..., description="해야 할 작업")
    assignee: str = Field(..., description="담당자")
    deadline: str = Field(..., description="마감일")
    priority: Priority = Field(..., description="우선순위")

class Decision(BaseModel):
    """결정 사항 모델이다."""
    topic: str = Field(..., description="결정 주제")
    decision: str = Field(..., description="내린 결정")
    rationale: str = Field(..., description="결정 이유")

class MeetingMinutes(BaseModel):
    """회의록 모델이다."""
    meeting_title: str = Field(..., description="회의 제목")
    date: str = Field(..., description="회의 날짜")
    attendees: List[str] = Field(..., description="참석자 목록")
    key_discussions: List[str] = Field(..., description="주요 논의 사항")
    decisions: List[Decision] = Field(..., description="결정 사항")
    action_items: List[ActionItem] = Field(..., description="실행 항목")
    next_meeting: str = Field(..., description="다음 회의 일정")

def create_meeting_minutes(meeting_transcript: str) -> MeetingMinutes:
    """회의 내용을 구조화된 회의록으로 변환한다."""
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "회의 내용을 구조화된 회의록으로 정리한다."},
            {"role": "user", "content": meeting_transcript}
        ],
        response_model=MeetingMinutes,
    )
    return response

# 사용 예시
transcript = """
2025년 1월 15일 신제품 기획 회의
참석자: 김팀장, 이과장, 박대리

김팀장: 오늘은 신제품 출시 일정에 대해 논의하겠습니다.
이과장: 시장 조사 결과 3월 출시가 적절할 것 같습니다.
박대리: 동의합니다. 경쟁사 동향도 고려하면 3월이 좋겠네요.

김팀장: 그럼 3월 15일을 목표로 하겠습니다. 매우 중요한 사안입니다.
이과장이 마케팅 계획을 다음 주까지 작성해 주세요. 우선순위는 높습니다.
박대리는 제품 사양서를 이번 주 금요일까지 완성해 주시기 바랍니다.

다음 회의는 1월 22일 오후 2시입니다.
"""

minutes = create_meeting_minutes(transcript)
print(f"=== {minutes.meeting_title} ===")
print(f"날짜: {minutes.date}")
print(f"참석자: {', '.join(minutes.attendees)}")

print("\n주요 논의:")
for discussion in minutes.key_discussions:
    print(f"  • {discussion}")

print("\n결정 사항:")
for decision in minutes.decisions:
    print(f"   {decision.topic}")
    print(f"    결정: {decision.decision}")

print("\n실행 항목:")
for action in minutes.action_items:
    print(f"   {action.task}")
    print(f"   담당: {action.assignee} | 마감: {action.deadline} | 우선순위: {action.priority.value}")

print(f"\n다음 회의: {minutes.next_meeting}")

=== 신제품 기획 회의 ===
날짜: 2025-01-15
참석자: 김팀장, 이과장, 박대리

주요 논의:
  • 신제품 출시 일정 논의
  • 시장 조사 결과에 따른 3월 출시 적정성

결정 사항:
   신제품 출시일
    결정: 3월 15일로 목표 설정

실행 항목:
   마케팅 계획 작성
   담당: 이과장 | 마감: 2025-01-22 | 우선순위: high
   제품 사양서 완성
   담당: 박대리 | 마감: 2025-01-20 | 우선순위: high

다음 회의: 2025-01-22 오후 2시


## 5. 유효성 검사 모범 사례

### 5.1 명확한 필드 설명 작성
LLM이 각 필드의 의미를 정확히 이해할 수 있도록 설명을 구체적으로 작성한다.

In [23]:
class GoodModel(BaseModel):
    """좋은 모델 예시다."""
    user_age: int = Field(
        ..., 
        ge=0, 
        le=150, 
        description="사용자의 실제 나이 (만 나이 기준, 0-150세)"
    )
    purchase_date: str = Field(
        ..., 
        description="구매 날짜 (YYYY-MM-DD 형식)"
    )

class BadModel(BaseModel):
    """나쁜 모델 예시다."""
    age: int  # 설명 없음, 검증 없음
    date: str  # 형식 불명확

### 5.2 기본값 활용
필수가 아닌 필드에는 합리적인 기본값을 제공한다.

In [24]:
from typing import Optional

class ConfigurableTask(BaseModel):
    """설정 가능한 작업 모델이다."""
    title: str = Field(..., description="작업 제목")
    priority: Priority = Field(Priority.MEDIUM, description="우선순위 (기본: 보통)")
    status: Status = Field(Status.TODO, description="상태 (기본: 할 일)")
    assignee: Optional[str] = Field(None, description="담당자 (선택사항)")
    tags: List[str] = Field(default_factory=list, description="태그 목록")

### 5.3 커스텀 검증 추가
복잡한 검증 로직은 `validator`를 사용한다.

In [25]:
from pydantic import BaseModel, Field, field_validator, ValidationInfo
from datetime import datetime

class Event(BaseModel):
    """이벤트 모델이다."""
    title: str = Field(..., description="이벤트 제목")
    start_date: str = Field(..., description="시작일 (YYYY-MM-DD)")
    end_date: str = Field(..., description="종료일 (YYYY-MM-DD)")
    
    @field_validator('start_date', 'end_date')
    @classmethod
    def validate_date_format(cls, v):
        """날짜 형식을 검증한다."""
        try:
            datetime.strptime(v, '%Y-%m-%d')
        except ValueError:
            raise ValueError(f"날짜 형식은 'YYYY-MM-DD'여야 한다: {v}")
        return v
    
    @field_validator('end_date')
    @classmethod
    def validate_end_after_start(cls, v: str, info: ValidationInfo) -> str:
        """종료일이 시작일 이후인지 확인한다."""
        if 'start_date' in info.data:
            start_dt = datetime.strptime(info.data['start_date'], '%Y-%m-%d')
            end_dt = datetime.strptime(v, '%Y-%m-%d')
            if end_dt < start_dt:
                raise ValueError("종료일은 시작일보다 빠를 수 없다.")
        return v